<a href="https://colab.research.google.com/github/vriadi/CS614-Gen-AI-with-LLMs/blob/main/Topic3_Training_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install accelerate datasets transformers
import torch, transformers, datasets, accelerate
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from accelerate.utils.memory import clear_device_cache
from datasets import get_dataset_split_names, load_dataset, get_dataset_config_names

In [ ]:
dataset = load_dataset("yelp_review_full")

In [ ]:
get_dataset_split_names("yelp_review_full")

['train', 'test']

In [ ]:
dataset["train"].features # features of the 'train' dataset

{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars']),
 'text': Value('string')}

In [ ]:
dataset["train"][17]

{'label': 4,
 'text': 'Old school.....traditional \\"mom \'n pop\\" quality and perfection. The best fish and chips you\'ll ever enjoy and equally superb fried shrimp. A great out of the way, non-corporate, vestige of Americana. You will love it.'}

In [ ]:
dataset["train"][-1]

{'label': 0,
 'text': 'I have been using this company for 11 months.  Ryan would come out every other week and do what he needed to be done. Very little was required as I have desert landscaping and pretty much maintained it myself.  Just needed blowing and general light cleanup.  Ryan was very thorough.\\nThen his brother, I believe, would  come out.  I would have to watch him as if I was not looking, it would be a pretty much \\"hit and run\\" literally.\\nI was expecting them Friday.  No show, no call as they never call when they cannot make it.  I have to call them and ask where they are or if they are coming.   Finally a text back saying they will be there Monday as they are running late.\\n\\nMonday comes and again no call no show.  I texted them...no answer...I was very upset at this point as I asked if they would trim a tree that is coming into my yard from a neighbor.  Mind you, there is very little to do here...no grass...no weeds..no plants to constantly trim.  I felt that I

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5, torch_dtype="auto")

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#### Create small subsets of the data using only 1000 examples for train and eval
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(1000)) # I dont want to use the entire dataset so I'm just extracting 1000 rows
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(1000))


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
small_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
small_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
!pip install evaluate
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments( # These are set as their default values (hyperparameters)
    report_to="none",
    num_train_epochs=5, # low hanging fruit to better the model
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10,
    output_dir="test_trainer",
    eval_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.606500,1.572286,0.372000


Epoch,Training Loss,Validation Loss,Accuracy
1,1.606500,1.572286,0.372000
2,1.384500,1.319840,0.472000
3,1.094300,1.116089,0.541000
4,0.937900,1.031960,0.580000
5,0.709100,1.002311,0.584000


TrainOutput(global_step=160, training_loss=1.2003173649311065, metrics={'train_runtime': 612.392, 'train_samples_per_second': 8.165, 'train_steps_per_second': 0.261, 'total_flos': 1315590712320000.0, 'train_loss': 1.2003173649311065, 'epoch': 5.0})

In [ ]:
training_args = TrainingArguments(
    report_to="none",
    output_dir="test_trainer",
    eval_strategy="epoch",
    num_train_epochs=5 ,
    learning_rate=1e-3, ### made learning rate bigger - bad choices
    per_device_train_batch_size=8,  ### smaller batch size - bad choices
    per_device_eval_batch_size=8,
    weight_decay=0.1,  ### 10x bigger weight decay (regularizer) - could be bad choices
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,  ### smaller gradient accum steps - bad choices
    ### cf smaller batch size also - bad choices
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10
)


In [ ]:
trainer = Trainer( # For huggingface you need to define the trainer
    model=model,
    args=training_args,
    train_dataset=small_train_dataset, # this is the dataset you're using
    eval_dataset=small_eval_dataset, # this is the dataset you're using
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

# Challenge

In [ ]:
training_args = TrainingArguments( # These are set as their default values (hyperparameters)
    report_to="none",
    num_train_epochs=10, # low hanging fruit to better the model
    learning_rate=2e-5,
    per_device_train_batch_size=20,
    per_device_eval_batch_size=20,
    weight_decay=0.1,
    warmup_ratio=0.1,
    gradient_accumulation_steps=2,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    logging_dir='./logs',
    logging_steps=10,
    output_dir="test_trainer",
    eval_strategy="epoch"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,3.241400,3.737739,0.205000
2,8.401300,9.509826,0.196000
3,4.972300,5.704241,0.196000
4,3.943700,5.358182,0.205000


# Notes

## Core Training Parameters

`num_train_epochs=10`
Number of times the model sees the entire training dataset. More epochs = longer training, higher chance of overfitting if too many.

`learning_rate=4e-2`
Step size for updating weights.
⚠️ For BERT, 4e-2 is extremely high — typical fine-tuning ranges are 1e-5 to 5e-5. At 4e-2, training will almost certainly diverge.

`per_device_train_batch_size=16`
Number of training samples per device (GPU/CPU) per step. Affects memory usage and training stability. Larger batch = smoother gradients, but more memory needed.

`per_device_eval_batch_size=16`
Same as above, but used during evaluation.

## Regularization & Stability

`weight_decay=0.1`
Adds penalty to large weights (L2 regularization). Helps prevent overfitting. For transformers, values around 0.01–0.1 are common.

`warmup_ratio=0.1`
Fraction of training steps spent increasing the learning rate linearly before decaying. Prevents instability at the start.
Example: If total steps = 10,000, then 1,000 warmup steps.

`gradient_accumulation_steps=2`
Accumulates gradients across multiple steps before updating weights. Useful when you want an effective larger batch size without using more GPU memory.
Effective batch size = train_batch_size × accumulation_steps × num_devices.

`adam_beta1=0.9`, `adam_beta2=0.999`, `adam_epsilon=1e-08`
Hyperparameters for AdamW optimizer:
* beta1: momentum term for past gradients.
* beta2: controls moving average of squared gradients (stability).
* epsilon: small value to avoid division by zero.
Usually, these defaults are fine for BERT.

## Logging & Output

`report_to="none"`
Disables reporting to tools like TensorBoard or WandB.

`logging_dir='./logs'`
Directory for logs (used if reporting is enabled).

`logging_steps=10`
Log training metrics every 10 steps.

`output_dir="test_trainer"`
Folder where checkpoints and model outputs are saved.

`eval_strategy="epoch"`
Runs evaluation at the end of each epoch. Alternative: "steps" (every N steps).